In [2]:
using Knet, Images, Colors, Base.Iterators, Random

In [3]:
wdict = Knet.load("dictionary.jld2", "dict")
UNK = wdict["<unk>"]
w2i(x) = get(wdict, x, UNK)

w2i (generic function with 1 method)

In [111]:
dataset4 = Knet.load("4dataset_total_vjump.jld2", "dataset")
summary(dataset4)

"24951-element Array{Tuple{NTuple{4,Array{Normed{UInt8,8},3}},String},1}"

In [112]:
dataset_instructions_or = []

open("instructions_unique_original.txt") do file
    for f in eachline(file)
    push!(dataset_instructions_or,f)    
    end
end
summary(dataset_instructions_or)

"21-element Array{Any,1}"

In [113]:
dataset_instructions = []

open("instructions_unique.txt") do file
    for f in eachline(file)
    push!(dataset_instructions,f)    
    end
end
summary(dataset_instructions)

"21-element Array{Any,1}"

In [114]:
images4 = []
images2 = []
images1 = []
labels = []

for data in dataset4
    (f1,f2, f3, f4), sent = data
    for i = 1:length(dataset_instructions_or)
        if(dataset_instructions_or[i]==sent)
            sent = dataset_instructions[i]
            break
        end
    end    
    f1 = Float32.(Gray.(colorview(RGB, permutedims(f1, (3,1,2)))))
    f2 = Float32.(Gray.(colorview(RGB, permutedims(f2, (3,1,2)))))
    f3 = Float32.(Gray.(colorview(RGB, permutedims(f3, (3,1,2)))))
    f4 = Float32.(Gray.(colorview(RGB, permutedims(f4, (3,1,2)))))
    #four_frames = cat(f1, f2, f3, f4; dims=3)
    three_frames = cat(f2-f1, f3-f2, f4-f3; dims=3)
    #both_frames = cat(f3, f4; dims=3)
    #last_frame = f4
    last_frame = f4-f3
    wordids = w2i.(split(sent))
    push!(labels, wordids)
    #push!(images4, four_frames)
    push!(images4,three_frames)
    #push!(images2,both_frames)
    push!(images1,last_frame)
end

In [115]:
image_batch4 = reshape(cat1d(images4...), (210,160,3,length(images4)))
summary(image_batch4)

"210×160×3×24951 Array{Float32,4}"

In [19]:
image_batch2 = reshape(cat1d(images2...), (210,160,2,length(dataset4)))
summary(image_batch2)

"210×160×2×14489 Array{Float32,4}"

In [116]:
image_batch1 = reshape(cat1d(images1...), (210,160,1,length(dataset4)))
summary(image_batch1)

"210×160×1×24951 Array{Float32,4}"

In [117]:
summary(labels)

"24951-element Array{Any,1}"

In [118]:
function seqbatch(sentences)
    batchsize = size(sentences, 1)
    maxlength = findmaxlength(sentences)
    for sent in sentences
        if(length(sent) < maxlength)
            for i = 1:maxlength-length(sent)
                push!(sent, 2)
            end
        end
    end
    reshape(cat1d(sentences...), (maxlength,length(labels)))
end

seqbatch (generic function with 1 method)

In [119]:
function findmaxlength(sentences)
    maxsize = 0
    count = 0
    for sent in sentences
        count = count + 1
        if(length(sent) > maxsize)
            maxsize = length(sent)
        end
    end
    return maxsize
end

findmaxlength (generic function with 1 method)

In [120]:
labels = seqbatch(labels)
summary(labels)

"8×24951 Array{Int64,2}"

In [121]:
data4 = Tuple{Array{Float32,3},Array{Int32,1}}[]
for i = 1:length(dataset4)
    x1 = image_batch4[:,:,:,i]
    x2 = labels[:,i]
    push!(data4, (x1,x2))
end
summary(data4)

InterruptException: InterruptException:

In [28]:
data2 = Tuple{Array{Float32,3},Array{Int32,1}}[]
for i = 1:length(dataset4)
    x1 = image_batch2[:,:,:,i]
    x2 = labels[:,i]
    push!(data2, (x1,x2))
end
summary(data2)
    

"14489-element Array{Tuple{Array{Float32,3},Array{Int32,1}},1}"

In [122]:
data1 = Tuple{Array{Float32,3},Array{Int32,1}}[]
for i = 1:length(dataset4)
    x1 = image_batch1[:,:,:,i]
    x2 = labels[:,i]
    push!(data1, (x1,x2))
end
summary(data1)
    

"24951-element Array{Tuple{Array{Float32,3},Array{Int32,1}},1}"

In [83]:
Knet.save("data3_diff.jld2", "dataset", data4)

In [31]:
Knet.save("data2.jld2", "dataset", data2)

In [123]:
Knet.save("data1_diff_wjump.jld2", "dataset", data1)

In [2]:
using Knet

In [10]:
tst = Knet.load("data1_diff_wjump.jld2", "dataset")
no_zero_dataset = Tuple{Array{Float32,3},Array{Int32,1}}[]
for tuple in tst
    x1, x2 = tuple
    if(sum(x1)!=0)
        push!(no_zero_dataset, (x1,x2))
    end
end
summary(no_zero_dataset)

"24326-element Array{Tuple{Array{Float32,3},Array{Int32,1}},1}"

In [12]:
Knet.save("data1_diff_wjump_nozero.jld2", "dataset", no_zero_dataset)